In [1]:
#setup libraries
import numpy as np
import pandas as pd
import os, re, glob, sys
from skimage import io, filters, util, segmentation, morphology, measure
import matplotlib.pyplot as plt
import imageio
from cellpose import models, plot
from pystackreg import StackReg
from scipy import stats
import tifffile

2022-01-04 13:50:30,297 [INFO] WRITING LOG OUTPUT TO /home/users/dane/.cellpose/run.log


In [2]:
pipeline_name = "PC" #python and cellpose
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'
plateID = 'AU02001'
#plateID = sys.argv[1]
well = "A1"
#well = sys.argv[2]

output_path = os.path.join(data_path+plateID,"Analysis",pipeline_name,"intermediate_files/")
transformation_path = os.path.join(output_path,"transformations")
subdirectories = sorted(glob.glob(os.path.join(data_path+plateID,well,"field_[1-9]")))[0:1]

Only create a job if there is not a registered red channel stack

In [3]:
flourescent_scaler = 255/4095 #rescale from 12 to 8 bits

for subdir in subdirectories:
    well = re.findall("/[A-Z][1-9]",subdir)[0]
    well = re.findall("[A-Z][1-9]", well)[0]
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    reg_filename = os.path.join(output_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    #metadata_filename = os.path.join(output_path,plateID+"_RGP_"+well+"_"+field_num+"_filenames.txt")
    # Only process the field-level image files if there is no stack of RGP files
    if not os.path.exists(reg_filename):
        print("registering R stack in "+subdir)
        #load and prepare red, green and phase channels. Scale for 8 bits but these are uint16 data types
        r_data_paths = glob.glob(os.path.join(subdir,"*_R_*m.tif"))
        r_time_slices = set()
        for data_paths in r_data_paths:
            r_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        g_data_paths = glob.glob(os.path.join(subdir,"*_G_*m.tif"))
        g_time_slices = set()
        for data_paths in g_data_paths:
            g_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        p_data_paths = glob.glob(os.path.join(subdir,"*_P_*m.tif"))
        p_time_slices = set()
        for data_paths in p_data_paths:
            p_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        complete_time_slices = r_time_slices & g_time_slices & p_time_slices
        r_data_paths_c = []
        g_data_paths_c = []
        p_data_paths_c = []
        for time_slice in complete_time_slices:
            r_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_R_"+well+"_"+field_num+"_"+time_slice+".tif"))
            g_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_G_"+well+"_"+field_num+"_"+time_slice+".tif"))
            p_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_P_"+well+"_"+field_num+"_"+time_slice+".tif"))
        img_r_ic = io.imread_collection(r_data_paths_c) # 3 dimensions : frames x width x height
        img_rs = np.stack(img_r_ic)*flourescent_scaler

        img_g_ic = io.imread_collection(g_data_paths_c) # 3 dimensions : frames x width x height
        img_gs = np.stack(img_g_ic)*flourescent_scaler

        img_p_ic = io.imread_collection(p_data_paths_c) # 3 dimensions : frames x width x height
        img_ps = np.stack(img_p_ic)
        
        #register the R stack using transformation
        sr = StackReg(StackReg.TRANSLATION)
        # register each frame to the previous (already registered) one
        tmats = sr.register_stack(img_rs, reference='previous')
        if not os.path.exists(transformation_path):
            os.makedirs(transformation_path)
        np.save(os.path.join(transformation_path,plateID+"_"+well+"_"+field+"_transformation_matrices.npy"), tmats)
        
        # transform stack using the tmats loaded from file
        img_rs_reg = sr.transform_stack(img_rs, tmats = tmats)
        img_gs_reg = sr.transform_stack(img_gs, tmats = tmats)
        img_ps_reg = sr.transform_stack(img_ps, tmats = tmats)

        img_c = np.stack([img_rs.astype('B'), img_gs.astype('B'), img_ps.astype('B')], axis = -1) 
    
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        io.imsave(reg_filename, img_rs_reg, plugin='tifffile')
        io.imsave(reg_filename.replace("_R_","_G_"), img_gs_reg, plugin='tifffile')
        io.imsave(reg_filename.replace("_R_","_P_"), img_ps_reg, plugin='tifffile')


segment each stack using cellpose


In [22]:
diameter = 16.5
flow_threshold = .4
mask_threshold=0
min_size=5
resample = False
cyto_expansion = 5
minutes_between_images = 30

#call cellpose on each image to segment the nuclei
# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=True, model_type='cyto2')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# will use channel R = 1 as nuclear channel only
channels = [0,1]

results = [] #collect results for all fields in the well
for subdir in subdirectories:
    well = re.findall("/[A-Z][1-9]",subdir)[0]
    well = re.findall("[A-Z][1-9]", well)[0]
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    reg_filename = os.path.join(output_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    l0_filename = os.path.join(data_path,plateID,'Analysis',pipeline_name,"intermediate_files",plateID+"_R_"+well+"_"+field+"_reg_stack.tif")
    img_rs_reg = io.imread(reg_filename)

    mask_images = []
    for i, image in enumerate(img_rs_reg):
        print("processing "+reg_filename+" index "+str(i))
        #Segment using the nuclear signal in the red channel
        image_rf = filters.median(image, footprint=morphology.disk(2))
        image_rf = image
        #io.imsave(reg_filename.replace("_R_","_R_"+str(i)+"_"), image_rf, plugin='tifffile')

        # create masks with cellpose 
        masks, flows, styles, diams = model.eval(image_rf,
                                          diameter=diameter,
                                         flow_threshold=flow_threshold,
                                        mask_threshold=mask_threshold,
                                         channels=channels,
                                                 min_size=min_size,
                                            resample = resample)
           
        #Save mask image
        mask_images.append(masks)
            

2022-01-04 15:00:04,888 [INFO] ** TORCH CUDA version installed and working. **
2022-01-04 15:00:04,889 [INFO] >>>> using GPU
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/PC/intermediate_files/AU02001_R_A1_1_reg_stack.tif index 0
2022-01-04 15:00:08,409 [INFO] ~~~ FINDING MASKS ~~~
2022-01-04 15:00:18,442 [INFO] >>>> TOTAL TIME 10.03 sec
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/PC/intermediate_files/AU02001_R_A1_1_reg_stack.tif index 1
2022-01-04 15:00:18,785 [INFO] ~~~ FINDING MASKS ~~~
2022-01-04 15:00:27,399 [INFO] >>>> TOTAL TIME 8.61 sec
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/PC/intermediate_files/AU02001_R_A1_1_reg_stack.tif index 2
2022-01-04 15:00:27,767 [INFO] ~~~ FINDING MASKS ~~~
2022-01-04 15:00:36,575 [INFO] >>>> TOTAL TIME 8.81 sec
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/PC/intermediate_files/AU02001_R_A1_1_reg_stack.tif index 3
2022-01-04 15:00:36,907 [INFO] ~~~ 

In [23]:
#concatenate all of the results and write to disk
#all_results = pd.concat(results)
#all_results.to_csv(output_path+plateID+'_'+well+'_level_0_test.csv')

In [24]:
#store the stack of masks
mask_filename = reg_filename.replace("_reg_stack.tif", "_masks_stack.png")
tifffile.imwrite(mask_filename, np.array(mask_images), imagej=True)

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 

In [71]:
masks

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)